In [2]:
import cv2
import numpy as np
import tensorflow as tf
import os
from datetime import datetime
import csv

In [3]:
# --- Configuration ---
MODEL_PATH = "Emotion_Recognition_Model.h5"
IMG_SIZE = 96  # must match training
use_face_cascade = True  # optional: set to False if you only want to run full frame

In [4]:
# --- Load Trained Model ---
model = tf.keras.models.load_model(MODEL_PATH)

In [5]:
# --- Frame Preprocessing Function ---
def preprocess_frame(frame, face_box=None):
    if face_box:
        (x, y, w, h) = face_box
        face = frame[y:y+h, x:x+w]
    else:
        face = frame

    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    face = face.astype("float32") / 255.0
    return np.expand_dims(face, axis=0)


In [6]:
# --- Face Detection ---
if use_face_cascade:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [7]:
# make dictionary to store captured images
save_dir = "captured_data"
os.makedirs(save_dir, exist_ok=True)

# File to store scores
csv_path = os.path.join(save_dir, "emotion_scores.csv")
if not os.path.exists(csv_path):
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["filename", "valence", "arousal", "timestamp"])

In [8]:
cap = cv2.VideoCapture(0)
print("Press 'q' to quit.")
print("Press 'c' to capture image and save prediction")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    display_frame = frame.copy()
    valence, arousal = 0.0, 0.0  # Default values

    if use_face_cascade:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_input = preprocess_frame(frame, (x, y, w, h))
            pred = model.predict(face_input)[0]
            valence, arousal = pred

            # Draw box and scores
            cv2.rectangle(display_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(display_frame, f"Valence: {valence:.2f}", (x, y - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            cv2.putText(display_frame, f"Arousal: {arousal:.2f}", (x, y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    else:
        face_input = preprocess_frame(frame)
        valence, arousal = model.predict(face_input)[0]

        # Display in corner
        cv2.putText(display_frame, f"Valence: {valence:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(display_frame, f"Arousal: {arousal:.2f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Real-Time Emotion Detection", display_frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('c'):
        # Save image with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"capture_{timestamp}.jpg"
        image_path = os.path.join(save_dir, filename)
        cv2.imwrite(image_path, frame)

        # Save prediction to CSV
        with open(csv_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([filename, round(valence, 4), round(arousal, 4), timestamp])
        print(f"Saved: {filename} with Valence={valence:.2f}, Arousal={arousal:.2f}")

cap.release()
cv2.destroyAllWindows()

Press 'q' to quit.
Press 'c' to capture image and save prediction
1/1 [==============================] - 0s 20ms/step
Saved: capture_20250618_172437.jpg with Valence=-0.21, Arousal=-0.33
1/1 [==============================] - 0s 18ms/step
Saved: capture_20250618_172444.jpg with Valence=-0.22, Arousal=-0.35
1/1 [==============================] - 0s 22ms/step


<h5>This is how the Final Output Looks:</h5>

In [ ]:
import pandas as pd

# Load your emotion scores CSV
df = pd.read_csv("captured_data/emotion_scores.csv")

# Select the target image
target_image = "capture_20250618_172444.jpg"

# Filter the dataframe to get the row for this image
row = df[df['filename'] == target_image]

if not row.empty:
    valence = row['valence'].values[0]
    arousal = row['arousal'].values[0]
else:
    print("Image not found in CSV.")

In [11]:
from IPython.display import Image, display, HTML

html_output = f"""
<h4>Final Output</h4>
<img src="captured_data/capture_20250618_172444.jpg" width="300"><br><br>
<b>valence:</b> {valence:.4f} <br>
<b>arousal:</b> {arousal:.4f}
"""

display(HTML(html_output))